In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
df_test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')



In [ ]:
# Consider only the features that are provided both in train and test datasets
dfx = df_train.copy()
dfx = dfx.drop(['density', 'biopsy', 'invasive', 'BIRADS', \
                             'difficult_negative_case'], axis=1)
dfx.head()

In [ ]:
# Change the type of id columns to string
dfx['site_id'] = dfx['site_id'].astype(str) 
dfx['patient_id'] = dfx['patient_id'].astype(str)
dfx['image_id'] = dfx['image_id'].astype(str)
dfx['machine_id'] = dfx['machine_id'].astype(str)
dfx.info()

In [ ]:
# We have unique numeric feature (age)

# Convert the type of implant feature to object (categorical feature)
dfx['implant'] = dfx['implant'].astype(object)

# label 
dfx['cancer'] = dfx['cancer'].astype(object)

dfx.info()

In [ ]:
# Number of patients
Number_patients = dfx.patient_id.nunique()
print('Number_patients = ', Number_patients) # 11913

# Explore missing values
dfx.isna().sum() 
# We have 37 missing values for age.

# Fill missing values with the mean, for age column
mean_age = dfx['age'].mean()

dfx['age'].fillna(value=mean_age, inplace=True)

## Normalize age column 
# Define scaler to set values between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

# subset of dataset
subset = dfx.filter(['age'], axis=1).copy()

# Normalize 
normalized_df = pd.DataFrame(scaler.fit_transform(subset), columns = subset.columns)

dfx['age'] = normalized_df['age']

dfx.head()
#dfx.iloc[:,5]

Label Encoding

laterality:
0: L (Left)
1: R (Right)
    
view:
0: AT (axillary tail)
1: CC (craniocaudal)
2: LM (latero-medial)
3: LMO (latero-medial oblique)
4: ML (medio-lateral)
5: MLO (mediolateral oblique)

In [ ]:
## One-Hot Encode categorical nominal features

#Select nominal variables
nominal_columns = ['laterality', 'view', 'implant']

# Get the dummies variable
dfx = pd.get_dummies(dfx, columns=nominal_columns, drop_first=True)

dfx.head()


In [ ]:
''' Explore the target '''

dfx['cancer'].value_counts() 

# => We have 53548 cases with cancer, and 1158 without cancer.

# We should correct the imbalanced target, by adding the pos_weight parameter to the 
# loss function.
